In [ ]:
import json
from datetime import datetime, timedelta

In [ ]:
WEEKDAY_TRANSLATION = {
    "Monday": "понедельник",
    "Tuesday": "вторник",
    "Wednesday": "среда",
    "Thursday": "четверг",
    "Friday": "пятница",
    "Saturday": "суббота",
    "Sunday": "воскресенье"
}

def get_academic_week(date_str, semester_start="2024-09-02"):
    """
    Вычисляет номер недели учебного года для заданной даты.
    """
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    semester_start_date = datetime.strptime(semester_start, "%Y-%m-%d")

    # Разница в днях между датой и началом семестра
    delta_days = (date_obj - semester_start_date).days

    if delta_days < 0:
        return None  # Дата раньше начала семестра

    academic_week = (delta_days // 7) + 1
    return academic_week

def print_schedule_for_day(occupied_rooms, building_name, date_str, room_name):
    """
    Выводит расписание для конкретной комнаты и здания на указанную дату.
    """
    # Определяем номер недели и день недели
    week = get_academic_week(date_str)

    if week is None:
        print(f"Дата {date_str} находится до начала семестра.")
        return

    print(week)

    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    weekday_en = date_obj.strftime("%A")  # Получаем день недели на английском
    weekday = WEEKDAY_TRANSLATION.get(weekday_en, "неизвестно")  # Переводим на русский

    if building_name not in occupied_rooms:
        print(f"Здание {building_name} не найдено.")
        return

    building = occupied_rooms[building_name]

    if room_name not in building:
        print(f"Комната {room_name} не найдена в {building_name}.")
        return

    room_schedule = building[room_name]

    # Фильтруем занятия по неделе и дню недели
    filtered_lessons = [
        lesson for lesson in room_schedule
        if lesson["weekday"] == weekday and lesson["week"] == week
    ]

    # Сортируем занятия по времени начала
    sorted_lessons = sorted(filtered_lessons, key=lambda lesson: datetime.strptime(lesson["begin_time"], "%H:%M"))

    # Группируем занятия по времени (чтобы избежать дублирования информации)
    schedule_dict = {}

    for lesson in sorted_lessons:
        begin_time = lesson["begin_time"]
        end_time = lesson["end_time"]

        key = (begin_time, end_time)  # Ключ для группировки

        if key not in schedule_dict:
            schedule_dict[key] = {
                "disciplines": set(),
                "groups": set(),
                "teachers": set()
            }

        schedule_dict[key]["disciplines"].add(lesson["discipline"])
        schedule_dict[key]["groups"].update(lesson["groups"])
        schedule_dict[key]["teachers"].update(lesson["teacher"])

    # Выводим расписание без дублей
    if schedule_dict:
        print(f"Расписание для {room_name} в {building_name} на {date_str} (учебная неделя {week}, {weekday}):\n")
        for (begin_time, end_time), data in schedule_dict.items():
            disciplines = ", ".join(data["disciplines"])
            groups = ", ".join(data["groups"])
            teachers = ", ".join(data["teachers"])

            print(f"Время: {begin_time} - {end_time}")
            print(f"Предмет(ы): {disciplines}")
            print(f"Группы: {groups}")
            print(f"Преподаватели: {teachers}")
            print("-" * 40)
    else:
        print(f"Нет занятий в комнате {room_name} в этот день ({date_str}).")


# Чтение JSON из файла
with open("occupied_rooms.json", "r", encoding="utf-8") as f:
    occupied_rooms = json.load(f)

# Передаем параметры для поиска
building_name = "14"
date = "2025-02-25"
room_name = "407-14"

# Выводим расписание
print_schedule_for_day(occupied_rooms, building_name, date, room_name)

In [ ]:
from datetime import datetime

def get_academic_week(date_str, semester_start="2024-09-02"):
    """
    Вычисляет номер недели учебного года для заданной даты.
    """
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    semester_start_date = datetime.strptime(semester_start, "%Y-%m-%d")

    # Разница в днях между датой и началом семестра
    delta_days = (date_obj - semester_start_date).days

    if delta_days < 0:
        return None  # Дата раньше начала семестра

    academic_week = (delta_days // 7) + 1
    return academic_week

def find_available_rooms(occupied_rooms, building_name, week, weekday, begin_time):
    available_rooms = []

    if building_name in occupied_rooms:
        building = occupied_rooms[building_name]

        # Проходим по всем комнатам в здании
        for room_name, room_schedule in building.items():
            # Проверяем каждое занятие в комнате
            is_room_available = True  # По умолчанию предполагаем, что комната свободна

            for lesson in room_schedule:
                if lesson["weekday"] == weekday and lesson["week"] == week:
                    # Преобразуем время начала и конца занятия в формат datetime
                    lesson_begin_time = datetime.strptime(lesson["begin_time"], "%H:%M")
                    lesson_end_time = datetime.strptime(lesson["end_time"], "%H:%M")
                    user_time = datetime.strptime(begin_time, "%H:%M")

                    # Если время начала передается как время в занятии, проверяем занятость
                    if lesson_begin_time <= user_time < lesson_end_time:
                        is_room_available = False
                        break

            # Если комната свободна на это время, добавляем её в список доступных
            if is_room_available:
                available_rooms.append(room_name)

    # Сортируем список доступных аудиторий в алфавитном порядке
    available_rooms = sorted(available_rooms)

    if available_rooms:
        print(f"В здании {building_name} на {weekday} недели {week}, в {begin_time} свободные аудитории:")
        for room in available_rooms:
            print(f"{room}, ")
    else:
        print(f"Нет свободных аудиторий в здании {building_name} на {weekday} недели {week}, в {begin_time}.")

building_name = "18"
week = 26
weekday = "понедельник"
begin_time = "09:45"

# Находим доступные аудитории
find_available_rooms(occupied_rooms, building_name, week, weekday, begin_time)
